In [1]:
#rdkit imports
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFMCS
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

#ml imports
#from sklearn.preprocessing import StandardScaler
#from sklearn import cross_validation
#from sklearn import cross_validation
#from sklearn.kernel_ridge import KernelRidge
#from sklearn.linear_model import Ridge, LinearRegression
#from sklearn.svm import SVR
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn.neural_network import MLPRegressor
#from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
#from sklearn.model_selection import GridSearchCV

#housekeeping imports
import pandas as pd
import matplotlib
import numpy as np
import scipy as sp

Maximum common substructure

In [2]:
# finds maximum common substructure among list of molecule objects
def get_mcs(input_df):      # gets a df containing SMILES strings of products
    
    mol_list = list(input_df['SMILES']) # converts product column to list

    #for i in range(len(mol_list)): # converts SMILES strings to molecule objects
        #if mol_list[i] = 'none'
        #mol_list[i] = Chem.MolFromSmiles(mol_list[i])

    #maximum_common_substructure = rdFMCS.FindMCS(mol_list)
    
    return mol_list #maximum_common_substructure

#Draw.MolToImage(maximum_common_substructure, size=(300, 300), kekulize=False, wedgeBonds=True)

Data cleaning

In [51]:
input_df = pd.read_csv('~/OneDrive/Documents/Python/CLASSES/DIRECT/metamoles/datasets/playground_df_cleaned_kegg_with_smiles.csv')

In [52]:
def input_data(input_df): #cleans input df and returns neccessary elements
    for index, row in input_df.iterrows():
        if row['SMILES'] == 'none':
            input_df.drop(index, inplace=True)            
        
    return input_df

In [54]:
def fingerprint_products(input_df): #fingerprints all products in a given df
    
    mol_list = []
    fp_list = []
    
    for index, row in input_df.iterrows():
        mol_list.append(Chem.rdmolfiles.MolFromSmiles(row['SMILES'])) #get mols from SMILES and add mols to list
        fp_list.append(FingerprintMols.FingerprintMol(Chem.rdmolfiles.MolFromSmiles(row['SMILES']))) #get fingerprints from mols and and fingerprints to list
        
    input_df.insert(6, column='Mol', value=mol_list)
    input_df.insert(7, column='Fingerprint', value= fp_list)
            
    return input_df

In [55]:
input_df2 = fingerprint_products(input_df) #build df containing new Mol and Fingerprint columns

split by enzyme

In [47]:
def split_by_enzyme(input_df):
    unique_enzymes = set(input_df['entry'].unique())
    
    enzyme_df_list = []
    
    for entry in unique_enzymes: #for each unique enzyme in the input dataframe...
        
        enzyme_df = pd.DataFrame(columns=input_df.columns) #...initialize a new dataframe with the same columns as the input dataframe...
        
        for index, row in input_df.iterrows(): #...iterate through the input dataframe...
            
            if row['entry'] == entry: #... and add product rows that correspond to the unique enzyme entry...
                enzyme_df.loc[index] = row
                
        enzyme_df_list.append(enzyme_df) #...then add the completed dataframe of unique enzyme products to a list
           
    return enzyme_df_list #return list of dataframes

In [204]:
def main(input_df):
    
    input_df = fingerprint_products(input_df)    #expand input df: generate mols from SMILES then generate fingerprints from mols, adding columns for each
    
    enzyme_df_list = split_by_enzyme(input_df)    #split expanded df by rows, grouped by enzyme entry (1.1.1.110 etc), into a list of dataframes
    
    master_df = pd.DataFrame()    #initialize output
    
    for enzyme_df in enzyme_df_list:    #loop through list of enzyme dataframes
        
        metric = sim_metric(input_df)    #get similarity matrix dataframe
        
        #for index, row in metric.iterrows():    #loop through similarity matrix
            #enzyme_df['Dist'].loc[index] = metric.[0]loc[index]
            
        #add averaged distance calculated from metric
    
    # concatenate enzyme df onto master_df     
    
    return master_df
        
        

similarity functions

In [57]:
def sim_i_j(row_i, row_j):
    """DocString"""
    return DataStructs.FingerprintSimilarity(row_i['Fingerprint'], row_j['Fingerprint'], metric=DataStructs.TanimotoSimilarity)

In [58]:
def sim_i_all(input_df, index_i, row_i, metric):
    """DocString"""
    for index_j, row_j in input_df.iterrows():
        if index_j < index_i: #skip redundant rows
            continue
        elif index_i == index_j: #autocorrelate rows
            metric.loc[index_i, index_j] = 1
        else:
            metric.loc[index_i, index_j] = sim_i_j(row_i, row_j) #fill matrix with calculated similarity at two positions at once 
            metric.loc[index_j, index_i] = metric.loc[index_i, index_j]
    return 

In [59]:
def sim_metric(input_df):
    """DocString"""
    metric = pd.DataFrame()
    for index_i, row_i in input_df.iterrows():
        sim_i_all(input_df, index_i, row_i, metric)
    return metric

In [62]:
dists = sim_metric(split_by_enzyme(input_df2)[0])
dists

,23,24,25,26
23,1.000000,0.507205,0.811594,0.495601
24,0.507205,1.000000,0.392713,0.720588
25,0.811594,0.392713,1.000000,0.358025
26,0.495601,0.720588,0.358025,1.000000


TypeError: 'type' object is not iterable